# Exploratory data analysis

This notebook contains exploratory data analysis focused on distributions and relationships between variables

In [1]:
import pandas as pd
import altair as alt
import re
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import numpy as np

In [2]:
melted_df = pd.read_csv("melted_data.csv")
df = pd.read_csv("data.csv")

## Understanding safety scores and its relationships to other variables

```A score from 0 to 100 indicating how safe an individual is. 0 is extremely high risk of injury, and 100 is very low risk of injury. The Safety Score is comprised of the twist speed, bend angle, and tilt speed.```

I am assuming that the lift rate is not part of safety score calculations based on the above.

### Distribution of scores

In [3]:
alt.Chart(melted_df).transform_density(
    'Average Safety Score', 
    groupby=['Stage'],
    as_=['Score', 'density']
).mark_line().encode(
    x='Score:Q',
    y='density:Q',
    color='Stage'
).properties(title="Safety Score distribution")

alt.Chart(...)

In [4]:
# just for curiosity purposes at this stage
stats.ttest_ind(df['Intervention Average Safety Score'], df['Baseline Average Safety Score'], nan_policy='omit')

Ttest_indResult(statistic=2.3968913068280266, pvalue=0.016921130811952838)

## Relationship to the other variables

There's a strong relationship between safety score and all other variables - including lift rate which is not part of score calculations. The correlations are quite similar in total and when looking at them by group.

In [5]:
melted_df.drop(['File Count'], axis=1).corr()

,Average Lift Rate,Average Max Forward Bend,Average Max Tilt Velocity,Average Safety Score,Average Twist Velocity
Average Lift Rate,1.000000,0.631283,0.101110,-0.531391,0.625421
Average Max Forward Bend,0.631283,1.000000,0.285163,-0.624532,0.139894
Average Max Tilt Velocity,0.101110,0.285163,1.000000,-0.866162,0.214834
Average Safety Score,-0.531391,-0.624532,-0.866162,1.000000,-0.497452
Average Twist Velocity,0.625421,0.139894,0.214834,-0.497452,1.000000


In [6]:
melted_df.drop(['File Count'], axis=1).groupby("Stage").corr()

Average Lift Rate  \
Stage                                                       
Baseline     Average Lift Rate                   1.000000   
             Average Max Forward Bend            0.621925   
             Average Max Tilt Velocity           0.085359   
             Average Safety Score               -0.546222   
             Average Twist Velocity              0.719658   
Intervention Average Lift Rate                   1.000000   
             Average Max Forward Bend            0.634665   
             Average Max Tilt Velocity           0.115799   
             Average Safety Score               -0.505488   
             Average Twist Velocity              0.565596   

                                        Average Max Forward Bend  \
Stage                                                              
Baseline     Average Lift Rate                          0.621925   
             Average Max Forward Bend                   1.000000   
             Average Max Tilt Velocity                  0.337489   
             Average Safety Score                      -0.679182   
             Average Twist Velocity                     0.297791   
Intervention Average Lift Rate                          0.634665   
             Average Max Forward Bend                   1.000000   
             Average Max Tilt Velocity                  0.270625   
             Average Safety Score                      -0.587244   
             Average Twist Velocity                     0.071569   

                                        Average Max Tilt Velocity  \
Stage                                                               
Baseline     Average Lift Rate                           0.085359   
             Average Max Forward Bend                    0.337489   
             Average Max Tilt Velocity                   1.000000   
             Average Safety Score                       -0.849977   
             Average Twist Velocity                      0.184507   
Intervention Average Lift Rate                           0.115799   
             Average Max Forward Bend                    0.270625   
             Average Max Tilt Velocity                   1.000000   
             Average Safety Score                       -0.887326   
             Average Twist Velocity                      0.245120   

                                        Average Safety Score  \
Stage                                                          
Baseline     Average Lift Rate                     -0.546222   
             Average Max Forward Bend              -0.679182   
             Average Max Tilt Velocity             -0.849977   
             Average Safety Score                   1.000000   
             Average Twist Velocity                -0.551257   
Intervention Average Lift Rate                     -0.505488   
             Average Max Forward Bend              -0.587244   
             Average Max Tilt Velocity             -0.887326   
             Average Safety Score                   1.000000   
             Average Twist Velocity                -0.473766   

                                        Average Twist Velocity  
Stage                                                           
Baseline     Average Lift Rate                        0.719658  
             Average Max Forward Bend                 0.297791  
             Average Max Tilt Velocity                0.184507  
             Average Safety Score                    -0.551257  
             Average Twist Velocity                   1.000000  
Intervention Average Lift Rate                        0.565596  
             Average Max Forward Bend                 0.071569  
             Average Max Tilt Velocity                0.245120  
             Average Safety Score                    -0.473766  
             Average Twist Velocity                   1.000000

### Understanding the score formula

How much of the score "formula" can we uncover from the averages (assuming some sort of linear model)? Looks like pretty much "all of it". It seems to be something along the lines of

`score = 120 -0.20 * bend - 0.3 * tilt - 0.6 * twist`

In [7]:
st_melted_df = melted_df.rename({
    "Average Safety Score": "score", 
    "Average Max Forward Bend": "forward_bend", 
    'Average Max Tilt Velocity':'tilt_velocity', 
    'Average Twist Velocity': 'twist_velocity',
    'Average Lift Rate': 'lift_rate'
}, axis=1)

model = ols('score ~ forward_bend + tilt_velocity + twist_velocity', data=st_melted_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                 1.254e+04
Date:                Fri, 27 May 2022   Prob (F-statistic):               0.00
Time:                        12:55:38   Log-Likelihood:                -450.04
No. Observations:                 476   AIC:                             908.1
Df Residuals:                     472   BIC:                             924.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        120.2107      0.298    403.599      0.000     119.625     120.796
forward_bend      -0.2027      0.003    -71.898      0.000      -0.208      -0.197
tilt_velocity     -0.2989      0.002   -127.383      0.000      -0.304      -0.294
twist_velocity    -0.5923      0.011    -55.960      0.000      -0.613      -0.571
==============================================================================
Omnibus:                       60.608   Durbin-Watson:                   1.962
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              164.875
Skew:                          -0.620   Prob(JB):                     1.58e-36
Kurtosis:                       5.603   Cond. No.                     1.21e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.21e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### How well does the lift rate alone predict the safety score?

In [8]:
model = ols('score ~ lift_rate', data=st_melted_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.282
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     186.5
Date:                Fri, 27 May 2022   Prob (F-statistic):           4.83e-36
Time:                        12:55:38   Log-Likelihood:                -1416.0
No. Observations:                 476   AIC:                             2836.
Df Residuals:                     474   BIC:                             2844.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     78.8316      0.825     95.598      0.000      77.211      80.452
lift_rate     -0.1107      0.008    -13.657      0.000      -0.127      -0.095
==============================================================================
Omnibus:                       54.292   Durbin-Watson:                   1.904
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               80.456
Skew:                          -0.773   Prob(JB):                     3.38e-18
Kurtosis:                       4.290   Cond. No.                         385.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
alt.Chart(melted_df).mark_point().encode(
    x='Average Lift Rate',
    y='Average Safety Score',
    column='Stage'
)

alt.Chart(...)

## Distributions of the other quantitative variables

In [10]:
vrs = ['Average Lift Rate', 'Average Max Forward Bend', 'Average Max Tilt Velocity', 'Average Twist Velocity']

charts = []

for v in vrs:  
    c = alt.Chart(melted_df).transform_density(
        v, 
        groupby=['Stage'],
        as_=['Metric', 'density']
    ).mark_line().encode(
        x=alt.X('Metric:Q', title=v),
        y='density:Q',
        color='Stage'
    ).properties(title= v + " distribution")
    
    charts.append(c)
    
alt.hconcat(*charts)

alt.HConcatChart(...)

How different are these distributions? Looks like the sensors result, on average, in both reduction of lift rates and the way workers bend (and, to some degree, the way workers twist).

In [11]:
results = []
for v in vrs:
    baseline = melted_df[melted_df['Stage'] == 'Baseline'][v]
    intervention = melted_df[melted_df['Stage'] == 'Intervention'][v]
    t = stats.ttest_ind(intervention, baseline, nan_policy='omit')
    results.append({
        'metric': v,
        'statistic': t[0],
        'p-value': t[1]
    })

pd.DataFrame(results).round(4)

,metric,statistic,p-value
0,Average Lift Rate,-3.1646,0.0017
1,Average Max Forward Bend,-7.6308,0.0000
2,Average Max Tilt Velocity,-0.2096,0.8340
3,Average Twist Velocity,1.6825,0.0931


## Distributions in baseline scenario

In [12]:
vrs = ['Average Lift Rate', 'Average Max Forward Bend', 'Average Max Tilt Velocity', 'Average Twist Velocity']

charts = []

baseline_melted_df = melted_df[melted_df['Stage'] == 'Baseline']

for v in vrs:  
    c = alt.Chart(baseline_melted_df).transform_density(
        v,         
        as_=['Metric', 'density']
    ).mark_line().encode(
        x=alt.X('Metric:Q', title=v),
        y='density:Q',
        #color='Stage'
    ).properties(title= v + " distribution")
    
    charts.append(c)
    
alt.hconcat(*charts)

alt.HConcatChart(...)

In [13]:
baseline_melted_df[vrs].mean()

Average Lift Rate            101.8008
Average Max Forward Bend      84.4992
Average Max Tilt Velocity     79.5412
Average Twist Velocity        20.1480
dtype: float64

In [14]:
alt.Chart(baseline_melted_df).transform_density(
    'Average Safety Score', 
    #groupby=['Stage'],
    as_=['Score', 'density']
).mark_line().encode(
    x='Score:Q',
    y='density:Q',
    #color='Stage'
).properties(title="Baseline Safety Score distribution")

alt.Chart(...)

In [18]:
alt.Chart(df).mark_point().encode(
    x='Baseline Average Lift Rate',
    y='Intervention Average Lift Rate'
)

alt.Chart(...)